In [2]:
from splinter import Browser
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pymongo
import pandas as pd
import requests
from sqlalchemy import create_engine
from config import postgrespw

In [2]:
# open browser
browser = Browser('chrome')
url = 'http://quotes.toscrape.com/'

In [3]:
# get author born, 
def quote_author(url):
    author_info = {}
    
    # request author html data
    authorPage = requests.get(url).text
    
    # parse response
    author = BeautifulSoup(authorPage, 'html.parser')
    
    # find author details
    author_info['name'] = author.find('h3', class_ = 'author-title').text
    author_info['birthday'] = author.find('span', class_ = 'author-born-date').text
    author_info['location'] = author.find('span', class_ = 'author-born-location').text
    author_info['description'] = author.find('div', class_ = 'author-description').text.strip()
    
    return author_info

In [4]:
# get tags
def get_tags(quote):
    tagList = []
    
    quoteTags = quote.find_all('a', class_='tag')
    
    for tag in quoteTags:
        tagList.append(tag.text)
    
    return tagList

In [5]:
# get quote info
def quote_data(quote):
    quote_info = {}
    quote_info['quote_text'] = quote.find('span', class_='text').text
    auth_url = urljoin(url, quote.find('a')['href'])
    
    # get author data
    quote_info['author'] = quote_author(auth_url)
    
    # get tag data
    quote_info['tag'] = get_tags(quote)
    
    return quote_info

In [6]:
# get list of quote boxes
def get_quoteList(pageNumber):
    quoteList = []
    
    # get html data from current page
    html = browser.html
    
    # use BeautifulSoup to parse html data
    soup = BeautifulSoup(html, 'html.parser')
    
    # find all quote blocks
    quotes = soup.find_all('div', class_='quote')
    
    # initialize quote ids
    quoteID = (pageNumber - 1) * 10
    
    # for each quote in the list retuned
    for quote in quotes:
        # increment quote id
        quoteID += 1
        
        # get all quote data returned as a dictionary
        quote_dic = quote_data(quote)
        quote_dic['_id'] = quoteID
        
        quoteList.append(quote_dic)
    
#         quoteList[0]: {_id: 1,
#                         quote_text: ,
#                         author:{name:
#                                 birthday:      
#                                 born:
#                                 description:}
#                         tags: []
#                       }
    
    return quoteList

In [7]:
# scrape everything
def scrap_everything(url):
    all_quotes = []
    
    first_iterations = True
    nextPage = True
    pageNumber = 1
    
    while nextPage == True:
        
        # if this is the first time through the while loop, navigate to url 
        if first_iterations == True:
            browser.visit(url)
            first_iterations = False
        else:
            pass
        
        # get all quote data from the page
        print(f'Scraping Page {pageNumber}')
        currentList = get_quoteList(pageNumber)
        all_quotes =  all_quotes + currentList
        
        try:
            browser.links.find_by_partial_text('Next').click()
            pageNumber += 1  
        except:
            print("Scraping Complete")
            nextPage = False
    
    return all_quotes
    

In [8]:
data = scrap_everything(url)

Scraping Page 1
Scraping Page 2
Scraping Page 3
Scraping Page 4
Scraping Page 5
Scraping Page 6
Scraping Page 7
Scraping Page 8
Scraping Page 9
Scraping Page 10
Scraping Page 11
Scraping Complete


In [9]:
browser.quit()

### Send data to MongoDB

In [10]:
# Initialize PyMongo to work with MongoDBs
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [11]:
# Define database and collection
db = client.quoteslist_db
collection = db.items

In [12]:
# Dictionary to be inserted as a MongoDB document
collection.insert_many(data)


### Move data from  MongoDB to put into postgres

In [3]:
def get_quotes_from_mongo():
    conn = 'mongodb://localhost:27017'
    client = pymongo.MongoClient(conn)
    db = client.quoteslist_db
    collection = db.items
    return collection

In [4]:
def normalize_quotes_data(docs):
    quotes_table = []
    authors = []
    authors_table = []
    tags_table = []
    for doc in docs.find({}):
#         print(f"normalizing the quote : [{doc['_id']}]")
        quote = {}
        quote['id'] = doc['_id']
        quote['quote_text'] = doc['quote_text']
        quote['author_name'] = doc['author']['name']
        quotes_table.append(quote)

        author = {}
        author['name'] = doc['author']['name']
        author['born'] =  doc['author']['birthday']
        author['birthplace'] =  doc['author']['location']
        author['description'] = doc['author']['description']
        if (author['name'] not in authors):
            authors_table.append(author)
            authors.append(author['name'])

        for tag in doc['tag']:
            tags_table.append({'quote_id':doc['_id'] , 'tag' : tag})    
            
    return (quotes_table , authors_table ,tags_table)

In [5]:
docs = get_quotes_from_mongo()
print(f' found {docs.count_documents({})} documents')

 found 110 documents


In [6]:
collection


NameError: name 'collection' is not defined

In [7]:
(quotes, authors, tags) = normalize_quotes_data(docs)
quotes_df = pd.DataFrame(quotes)
author_df = pd.DataFrame(authors)
tags_df = pd.DataFrame(tags)

In [8]:
quotes_df

,id,quote_text,author_name
0,1,“The world as we have created it is a process ...,Albert Einstein
1,2,"“It is our choices, Harry, that show what we t...",J.K. Rowling
2,3,“There are only two ways to live your life. On...,Albert Einstein
3,4,"“The person, be it gentleman or lady, who has ...",Jane Austen
4,5,"“Imperfection is beauty, madness is genius and...",Marilyn Monroe
...,...,...,...
105,106,“You never really understand a person until yo...,Harper Lee
106,107,“You have to write the book that wants to be w...,Madeleine L'Engle
107,108,“Never tell the truth to people who are not wo...,Mark Twain
108,109,"“A person's a person, no matter how small.”",Dr. Seuss


In [9]:
author_df

,name,born,birthplace,description
0,Albert Einstein,"March 14, 1879","in Ulm, Germany","In 1879, Albert Einstein was born in Ulm, Germ..."
1,J.K. Rowling,"July 31, 1965","in Yate, South Gloucestershire, England, The U...",See also: Robert GalbraithAlthough she writes ...
2,Jane Austen,"December 16, 1775","in Steventon Rectory, Hampshire, The United Ki...",Jane Austen was an English novelist whose work...
3,Marilyn Monroe,"June 01, 1926",in The United States,Marilyn Monroe (born Norma Jeane Mortenson; Ju...
4,André Gide,"November 22, 1869","in Paris, France",André Paul Guillaume Gide was a French author ...
5,Thomas A. Edison,"February 11, 1847","in Milan, Ohio, The United States","Thomas Alva Edison was an American inventor, s..."
6,Eleanor Roosevelt,"October 11, 1884",in The United States,Anna Eleanor Roosevelt was an American politic...
7,Steve Martin,"August 14, 1945","in Waco, Texas, The United States","Stephen Glenn ""Steve"" Martin is an American ac..."
8,Bob Marley,"February 06, 1945","in Nine Mile, Saint Ann, Jamaica","Robert ""Bob"" Nesta Marley OM was a Jamaican si..."
9,Dr. Seuss,"March 02, 1904","in Springfield, MA, The United States",Theodor Seuss Geisel was born 2 March 1904 in ...


In [10]:
tags_df

,quote_id,tag
0,1,change
1,1,deep-thoughts
2,1,thinking
3,1,world
4,2,abilities
...,...,...
241,107,writing
242,108,truth
243,109,inspirational
244,110,books


In [29]:
user_name = 'postgres'
password = 'password'
connection_string = f"postgres:{password}@localhost:5432/scrape_db"
engine = create_engine(f'postgresql://{connection_string}')

In [30]:
quotes_script = '''
    create table quotes(     id INTEGER PRIMARY KEY,    
    quote_text varchar(1500),
    author_name varchar(32))
'''

tags_script = '''
    create table tags(    quote_id INTEGER,    
    tag varchar(32))
'''

author_script = '''
    create table author( name varchar(32) PRIMARY KEY,    
    born varchar(32), 
    birthplace varchar(120),
    description varchar(10000))
'''

In [31]:
tables = {'quotes' : quotes_script.strip(), 
          'tags' : tags_script.strip(),
          'author' : author_script.strip()
         }

In [32]:
for table in tables.keys():
    print(f'dropping the table {table} if it already exists...')
    engine.execute(f'drop table IF EXISTS {table}')

dropping the table quotes if it already exists...
dropping the table tags if it already exists...
dropping the table author if it already exists...


In [33]:
for table , script in tables.items():
    print(f'creating the table {table}...')
    engine.execute(f'{script}')
    

creating the table quotes...
creating the table tags...
creating the table author...


In [34]:
engine.table_names()

['quotes', 'tags', 'author']

In [35]:
quotes_df.to_sql(name='quotes', con=engine, if_exists='append', index=False)

In [36]:
tags_df.to_sql(name='tags', con=engine, if_exists='append', index=False)

In [37]:
author_df.to_sql(name='author', con=engine, if_exists='append', index=False)